In [ ]:
import torch
import heapq
import pickle
import time
import json
import numpy as np
import pandas as pd
import os
import cProfile
import pstats
from pathlib import Path
from tqdm.auto import tqdm
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from psutil import cpu_count

SAE_PATH      = Path('out/sae_65k_lambda26_ramp30/sae_final.pt')
TOP_N         = 50
DUMP_DIR      = Path('feature_dumps')
DUMP_DIR.mkdir(exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)
# ---- Load SAE (replace with your own class/loader) ----
from model import SAE
state_dict, config = torch.load(SAE_PATH, map_location=device).values()
sae = SAE(config['input_size'],config['hidden_size']).to(device).to(torch.bfloat16)
# Fix for "_orig_mod" prefix in state dict keys
fixed_state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
sae.load_state_dict(fixed_state_dict)

n_features = sae.encode.out_features if hasattr(sae.encode,'out_features') else sae.n_features
print(f'Loaded SAE with {n_features} features')

def count_dead_features(sample_iter, sample_tokens=10_000_000):
    """Returns a boolean tensor of shape (n_features,) where True == dead."""
    fired = torch.zeros(n_features, dtype=torch.bool, device=device)
    seen  = 0
    for toks in tqdm(sample_iter, total=sample_tokens//len(next(iter(sample_iter)))):
        toks = toks.to(device)
        acts = sae.encode(toks) > 0  # bool mask of activations
        fired |= acts.any(dim=0)
        seen  += toks.size(0)
        if seen >= sample_tokens:
            break
    dead_mask = ~fired.cpu()
    print(f"Dead features: {dead_mask.sum().item()} / {n_features} ({dead_mask.float().mean()*100:.2f}%)")
    return dead_mask

# GPU optimizations
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision('high')

# Constants
MODEL_NAME = "allenai/OLMo-2-1124-7B-Instruct"
BATCH_SIZE = 256
LAYER_OFFSET = -1
TOP_N = 50
device = "cuda"
DUMP_DIR = Path("./results")
os.makedirs(DUMP_DIR, exist_ok=True)

def main():
    # Load tokenizer + *half* model config (bf16, compiled)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    cfg = AutoModelForCausalLM.from_pretrained(MODEL_NAME).config
    cfg.num_hidden_layers //= 2  # half-model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        config=cfg,
        torch_dtype=torch.bfloat16,
        attn_implementation="sdpa",
        device_map=device
    ).eval()
    model = torch.compile(model, mode="reduce-overhead")

    # Load dataset
    dataset_iter = load_dataset(
        "HuggingFaceFW/fineweb",
        name="sample-10BT",
        split="train",
        streaming=False,
        num_proc=cpu_count(),
    ).shuffle()

    def residual_stream_iter(text_iter, batch=BATCH_SIZE):
        buf = []
        for record in text_iter:
            buf.extend(tokenizer(record["text"]).input_ids)
            while len(buf) >= batch:
                toks = torch.tensor(buf[:batch]).to(device)
                buf = buf[batch:]
                with torch.inference_mode():
                    outs = model(toks.unsqueeze(0), output_hidden_states=True)
                    resid = outs.hidden_states[LAYER_OFFSET].squeeze(0)  # (T, d)
                yield toks, resid  # feed straight to SAE

    def mine_top_tokens_and_dead(data_iter,
                              top_n=TOP_N,
                              target_tokens=100_000):
        """
        • Keeps the TOP-N strongest (activation, token) pairs per feature.
        • Tracks which features ever fire to flag the 'dead' ones.
        • Stops after `target_tokens` have been processed.
        """
        n_features = sae.encode.weight.shape[0]
        
        # Pre-allocate all buckets with empty heaps
        buckets = [[] for _ in range(n_features)]
        fired = torch.zeros(n_features, dtype=torch.bool, device=device)
        seen_toks = 0
        
        # Process batches with lighter progress indicator 
        start_time = time.time()
        batch_count = 0
        
        for toks, resid in data_iter:
            batch_count += 1
            if batch_count % 10 == 0:
                elapsed = time.time() - start_time
                tokens_per_sec = seen_toks / elapsed if elapsed > 0 else 0
                print(f"\rProcessed {seen_toks} tokens ({tokens_per_sec:.1f} tok/s)", end="")
                
            with torch.inference_mode():
                # Compute activations
                acts = sae.encode(resid)
                fired |= (acts > 0).any(dim=0)
                
                # Get top values and indices
                values, idx = acts.topk(1, dim=0)
                
                # Transfer to CPU in one batch  
                values_cpu = values[0].to(torch.float32).detach().cpu().numpy()
                indices_cpu = idx[0].detach().cpu().numpy()
                token_ids = toks[indices_cpu].cpu().numpy()
                
                # Process features in chunks for better performance
                for f in range(n_features):
                    val, tok_id = float(values_cpu[f]), int(token_ids[f])
                    heap = buckets[f]
                    if len(heap) < top_n:
                        heapq.heappush(heap, (val, tok_id))
                    elif val > heap[0][0]:
                        heapq.heapreplace(heap, (val, tok_id))
            
            seen_toks += toks.numel()
            if seen_toks >= target_tokens:
                break
        
        print(f"\nProcessed {seen_toks} tokens in {time.time() - start_time:.2f}s")
        
        # Post-process
        dead_mask = ~fired.cpu().numpy()  # Convert directly to numpy
        
        # Get unique token IDs for batch decoding
        unique_token_ids = set()
        for heap in buckets:
            for _, tok_id in heap:
                unique_token_ids.add(tok_id)
        
        # Convert set to list for batch decoding
        unique_token_list = list(unique_token_ids)
        decoded_tokens = tokenizer.batch_decode([[t] for t in unique_token_list])
        
        # Create mapping of token ID to decoded text
        token_id_to_text = {unique_token_list[i]: decoded_tokens[i] for i in range(len(unique_token_list))}
        
        # Create the final result with native Python types
        decoded = []
        for heap in buckets:
            feature_results = []
            for val, tok_id in sorted(heap, key=lambda x: -x[0]):  # Sort directly here
                feature_results.append((float(val), token_id_to_text[tok_id]))
            decoded.append(feature_results)  # Already sorted
        
        print(f"Dead features: {dead_mask.sum()} / {n_features} "
              f"({dead_mask.sum()/n_features*100:.2f}%)")
        
        return decoded, dead_mask

    data_iter = residual_stream_iter(dataset_iter)
    target_tokens = 50_000_000
    with torch.inference_mode():
        top_buckets, dead_mask = mine_top_tokens_and_dead(
            data_iter,
            top_n=50,
            target_tokens=50_000_000
        )

    # Save results - using faster formats where possible
    # Use pickle for faster serialization
    with open(DUMP_DIR / f"top_tokens_{target_tokens}.pkl", "wb") as f:
        pickle.dump(top_buckets, f)
    
    with open(DUMP_DIR / f"top_tokens_{target_tokens}.json", "w") as f:
        json.dump(top_buckets, f)
    
    np.save(DUMP_DIR / f"dead_features_{target_tokens}.npy", dead_mask)
    
    pd.Series(dead_mask).to_csv(DUMP_DIR / f"dead_features_{target_tokens}.csv", index=False)

# Run with profiling
if __name__ == "__main__":
    profiler = cProfile.Profile()
    profiler.enable()
    
    main()
    
    profiler.disable()
    
    # Save stats to a file
    stats = pstats.Stats(profiler)
    stats.sort_stats('cumtime')
    stats.dump_stats('profile_results.prof')
    
    print("\n\n--- Profiling Results ---")
    stats.sort_stats('cumtime').print_stats(20)
    
    print("\n\n--- Profiling Results by Function Calls ---")
    stats.sort_stats('calls').print_stats(20)

Device: cuda


Exception ignored When destroying _lsprof profiler:
Traceback (most recent call last):
  File "/tmp/ipykernel_1253868/3328758826.py", line 210, in <module>
RuntimeError: Cannot install a profile function while another profile function is being installed


Loaded SAE with 65536 features


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.57it/s]
Some weights of the model checkpoint at allenai/OLMo-2-1124-7B-Instruct were not used when initializing Olmo2ForCausalLM: ['model.layers.16.mlp.down_proj.weight', 'model.layers.16.mlp.gate_proj.weight', 'model.layers.16.mlp.up_proj.weight', 'model.layers.16.post_attention_layernorm.weight', 'model.layers.16.post_feedforward_layernorm.weight', 'model.layers.16.self_attn.k_norm.weight', 'model.layers.16.self_attn.k_proj.weight', 'model.layers.16.self_attn.o_proj.weight', 'model.layers.16.self_attn.q_norm.weight', 'model.layers.16.self_attn.q_proj.weight', 'model.layers.16.self_attn.v_proj.weight', 'model.layers.17.mlp.down_proj.weight', 'model.layers.17.mlp.gate_proj.weight', 'model.layers.17.mlp.up_proj.weight', 'model.layers.17.post_attention_layernorm.weight', 'model.layers.17.post_feedforward_layernorm.weight', 'model.layers.17.self_attn.k_norm.weight', 'model.layers.17.self_attn.k_proj.weight', 'model.layers.17

Processed 49999104 tokens (2243.5 tok/s)
Processed 50000128 tokens in 22286.87s
Dead features: 2336 / 65536 (3.56%)


--- Profiling Results ---
         13146823416 function calls (13117207214 primitive calls) in 22375.153 seconds

   Ordered by: cumulative time
   List reduced from 15830 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    4.105    4.105 22376.231 22376.231 /tmp/ipykernel_1253868/3328758826.py:66(main)
        1 12310.012 12310.012 22294.526 22294.526 /tmp/ipykernel_1253868/3328758826.py:101(mine_top_tokens_and_dead)
   585940 8301.520    0.014 8301.520    0.014 {method 'cpu' of 'torch._C.TensorBase' objects}
   195314   13.252    0.000  870.551    0.004 /tmp/ipykernel_1253868/3328758826.py:89(residual_stream_iter)
12803479658/12803373515  705.880    0.000  705.909    0.000 {built-in method builtins.len}
585939/390626    2.018    0.000  338.463    0.001 /home/henry/.local/lib/python3.10/site-packages/torch

In [4]:
import asyncio, pickle, pandas as pd, instructor
from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from tqdm.asyncio import tqdm
from dotenv import load_dotenv
import os
load_dotenv()
client = instructor.from_openai(
    AsyncOpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.environ.get("OPENROUTER_API_KEY"))
)

class FeatureLabel(BaseModel):
    label: str = Field(..., description="≤5-word noun phrase or 'uncertain'")
    chain_of_thought: str = Field(..., description="One-sentence rationale")


def make_prompt(tokens: list[tuple]) -> str:
    token_str = ", ".join(str(tok) for _, tok in tokens)

    return (
        "You will receive several tokens that all activate *the same* hidden feature.\n\n"
        f"Tokens: {token_str}\n\n"
        "Return **only** the following JSON object (no additional text):\n"
        "{\n"
        '  "label": "<≤5-word noun phrase, or \'uncertain\', or \'punctuation\'>",\n'
        '  "chain_of_thought": "<one concise sentence explaining why>"\n'
        "}\n\n"
        "• Use \"uncertain\" if no coherent feature emerges.\n"
        "• Use \"punctuation\" if the tokens are mostly punctuation or formatting marks.\n"
        "• Do not output anything except that JSON object."
    )


top_buckets = pickle.load(open("results/top_tokens_50m.pkl", "rb"))
sem = asyncio.Semaphore(20)

async def label_one(idx, tokens):
    prompt = make_prompt(tokens)
    async with sem:
        fl = await client.chat.completions.create(
            model="google/gemini-2.0-flash-001",
            response_model=FeatureLabel,
            temperature=0.6,
            messages=[{"role":"user","content":prompt}],
        )
    return {
        "feature_id": idx,
        "label": fl.label,
        "chain_of_thought": fl.chain_of_thought,
        "tokens": tokens,
    }

async def main():
    tasks = [asyncio.create_task(label_one(i, t)) for i, t in enumerate(top_buckets)]
    return await tqdm.gather(*tasks)

rows = await main()

df = pd.DataFrame(rows).sort_values("feature_id")
df.to_csv("results/feature_labels.csv", index=False)
df.head()


100%|██████████| 65536/65536 [37:15<00:00, 29.31it/s]  


,feature_id,label,chain_of_thought,tokens
0,0,stop word,"The tokens are all common English stop words, ...","[(2.8125, the), (2.671875, ane), (2.59375, i..."
1,1,Worth,"The word 'worth' appears frequently, suggestin...","[(3.234375, can), (3.140625, Worth), (3.0312..."
2,2,punctuation,The tokens are all punctuation marks.,"[(4.5, .\n), (4.1875, \n), (4.125, \n), (3.828..."
3,3,uncertain,"The tokens appear to be a mix of common words,...","[(3.0625, _SCHEMA), (2.859375, _SCHEMA), (2.81..."
4,4,atheism,The tokens frequently co-occur with variations...,"[(2.09375, athe), (2.03125, Evans), (2.01562..."


In [8]:
#make a list of the labels that are not "uncertain"
uncertain_labels = df[df["label"] == "uncertain"]["label"].tolist()
#make a list of the labels that are not "punctuation"
punctuation_labels = df[df["label"] == "punctuation"]["label"].tolist()
#make a list of the labels that are not "uncertain" or "punctuation"
other_labels = df[~df["label"].isin(["uncertain", "punctuation"])]["label"].tolist()
print(other_labels)

#make a list of the labels that are not "uncertain" or "punctuation" or "other"



['stop word', 'Worth', 'atheism', 'willingness', 'Reading', 'zombie', 'grammatical function words', 'movement and transition', 'Profession', 'US President', 'Emphasis', 'grammatical function words', 'social decision-making', 'matching', 'monitoring and utilization', 'first-person pronoun', 'June', 'supply chain advantages', 'connecting words', 'places and events', 'Foreign origin or location', 'common language elements', 'Sports League or Tournament', 'Product Quality and Support', 'Possession and reference', 'family and inheritance', 'students learning', 'common English words', 'Business locations and names', 'softening', 'stop words and verbs', 'personalization', 'Negation', 'for', 'dog', 'life', 'state of completion', 'database', 'arrangement of physical areas', 'Trump', 'visit', 'spatial relationships', 'Mixed martial arts', 'Following item in sequence', 'posting activity', 'Upward trend', 'location or condition', 'closing or securing', 'performance enhancement', 'auxiliary verb', 

In [1]:
# Import required libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "allenai/OLMo-2-1124-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

default_system_prompt = "You are OLMo 2, a helpful and harmless AI Assistant built by the Allen Institute for AI."

def chat_with_olmo(user_message, system_prompt=default_system_prompt, history=None):
    if history is None:
        history = []
    
    # Create messages list as before
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    for h in history:
        messages.append(h)
    messages.append({"role": "user", "content": user_message})
    
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize with explicit attention mask
    inputs = tokenizer(
        formatted_prompt, 
        return_tensors="pt",
        padding=True,
        return_attention_mask=True  # Explicitly request attention mask
    ).to(device)
    
    # Generate with attention mask
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,  # Pass the attention mask
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Rest of the function remains the same
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    assistant_response = full_response.split("<|assistant|>")[-1].split("<|endoftext|>")[0].strip()
    
    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": assistant_response})
    
    return assistant_response, history

# Example usage
conversation_history = []

# First exchange
user_input = "What capabilities do you have as an AI assistant?"
response, conversation_history = chat_with_olmo(user_input, history=conversation_history)
print(f"User: {user_input}")
print(f"OLMo: {response}")
print("-" * 50)

# Follow-up question (demonstrating conversation history)
user_input = "Can you give me an example of how you might help with coding?"
response, conversation_history = chat_with_olmo(user_input, history=conversation_history)
print(f"User: {user_input}")
print(f"OLMo: {response}")

/home/henry/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 49.44it/s]


User: What capabilities do you have as an AI assistant?
OLMo: As OLMo 2, I have a wide range of capabilities! Some of the key ones include understanding and generating human-like text, answering questions across many topics, and assisting with a variety of tasks such as scheduling, language translation, summarizing texts, and much more. I'm designed to continuously improve my responses based on the feedback I receive, so I'm here to learn and help you better over time. How can I assist you today?
--------------------------------------------------
User: Can you give me an example of how you might help with coding?
OLMo: Certainly! While I don't write or debug code myself, I can certainly help guide you through concepts, explain coding principles, or help you understand a particular problem you might be facing with coding. For example, if you're trying to solve a problem using Python and you're unsure about the syntax, I can provide you with the correct syntax or direct you to resources 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from model import SAE
from pathlib import Path
import pandas as pd
import ast

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "allenai/OLMo-2-1124-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
default_system_prompt = "You are OLMo 2, a helpful and harmless AI Assistant built by the Allen Institute for AI."

# Load feature max values from CSV
csv_path = "results_65k_lambda26_ramp30/feature_labels.csv"
df = pd.read_csv(csv_path)
feature_max = df["tokens"].apply(lambda x: ast.literal_eval(x)[0][0] if x and ast.literal_eval(x) else 0.0).values
feature_max_tensor = torch.tensor(feature_max, dtype=torch.float32, device=device)

# Load the SAE model
sae_path = Path('out/sae_65k_lambda26_ramp30/sae_final.pt')
state_dict, config = torch.load(sae_path, map_location=device).values()
sae = SAE(config['input_size'], config['hidden_size']).to(device).to(torch.bfloat16).eval()
fixed_state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
sae.load_state_dict(fixed_state_dict)

# Load first half model
cfg_first_half = AutoConfig.from_pretrained(model_name)
cfg_first_half.num_hidden_layers //= 2
first_half_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=cfg_first_half,
    torch_dtype=torch.bfloat16,
    device_map=device
).eval()

# Load second half model
cfg_second_half = AutoConfig.from_pretrained(model_name)
cfg_second_half.num_hidden_layers -= cfg_first_half.num_hidden_layers
second_half_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=cfg_second_half,
    torch_dtype=torch.bfloat16,
    device_map=device
).eval()

def chat_with_steered_olmo(user_message, system_prompt=default_system_prompt, history=None, feature_idx=758, scale=10.0):
    if history is None:
        history = []
    
    # Create messages list
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    for h in history:
        messages.append(h)
    messages.append({"role": "user", "content": user_message})
    
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize with explicit attention mask
    inputs = tokenizer(
        formatted_prompt, 
        return_tensors="pt",
        padding=True,
        return_attention_mask=True
    ).to(device)
    
    # Create a tensor to store generated token IDs
    generated = inputs.input_ids.clone()
    attention_mask = inputs.attention_mask.clone()
    
    # Number of tokens to generate
    max_new_tokens = 512
    
    for _ in range(max_new_tokens):
        # Get first half outputs
        with torch.inference_mode():
            first_half_outputs = first_half_model(
                input_ids=generated,
                attention_mask=attention_mask,
                output_hidden_states=True
            )
            hidden_states = first_half_outputs.hidden_states[-1]
            
            # Apply SAE steering
            B, S, D = hidden_states.shape
            flat = hidden_states.reshape(-1, D)
            f = sae.encode(flat)
            recon = sae.decode(f)
            error = flat - recon
            
            # CLAMP feature
            target_val = scale * feature_max_tensor[feature_idx]
            f[:, feature_idx] = target_val
            recon_mod = sae.decode(f)
            steered_hidden = (recon_mod + error).reshape(B, S, D)
            
            # Get second half outputs
            second_half_outputs = second_half_model(
                inputs_embeds=steered_hidden,
                attention_mask=attention_mask
            )
            
            # Get logits for the next token
            next_token_logits = second_half_outputs.logits[:, -1, :]
            
            # Temperature sampling
            next_token_logits = next_token_logits / 0.7
            
            # Top-p (nucleus) sampling
            sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
            cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
            
            # Remove tokens with cumulative probability above 0.9
            sorted_indices_to_remove = cumulative_probs > 0.9
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            
            indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
            next_token_logits[indices_to_remove] = -float('Inf')
            
            # Sample from the filtered distribution
            probs = torch.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # Append the new token to the sequence
            generated = torch.cat([generated, next_token], dim=-1)
            attention_mask = torch.cat([attention_mask, attention_mask.new_ones((attention_mask.shape[0], 1))], dim=-1)
            
            # Check for EOS token
            if next_token[0, 0].item() == tokenizer.eos_token_id:
                break
    
    full_response = tokenizer.decode(generated[0], skip_special_tokens=False)
    assistant_response = full_response.split("<|assistant|>")[-1].split("<|endoftext|>")[0].strip()
    
    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": assistant_response})
    
    return assistant_response, history

# Example usage
conversation_history = []
# First exchange
user_input = "What capabilities do you have as an AI assistant?"
response, conversation_history = chat_with_steered_olmo(
    user_input, 
    history=conversation_history,
    feature_idx=758,
    scale=1
)
print(f"User: {user_input}")
print(f"Steered OLMo: {response}")
print("-" * 50)
# Follow-up question
user_input = "Can you give me an example of how you might help with coding?"
response, conversation_history = chat_with_steered_olmo(
    user_input, 
    history=conversation_history,
    feature_idx=758,
    scale=10.0
)
print(f"User: {user_input}")
print(f"Steered OLMo: {response}")

/home/henry/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s]
Some weights of the model checkpoint at allenai/OLMo-2-1124-7B-Instruct were not used when initializing Olmo2ForCausalLM: ['model.layers.16.mlp.down_proj.weight', 'model.layers.16.mlp.gate_proj.weight', 'model.layers.16.mlp.up_proj.weight', 'model.layers.16.post_attention_layernorm.weight', 'model.layers.16.post_feedforward_layernorm.weight', 'model.layers.16.self_attn.k_norm.weight', 'model.layers.16.self_attn.k_proj.weight', 'model.layers.16.self_attn.o_proj.weight', 'model.layers.16.self_attn.q_norm.weight', 'model.layers.16.self_attn.q_proj.weight', 'model.layers.16.self_attn.v_proj.weight', 'model.layers.17.mlp.down_proj.weight', 'model.layers.17.mlp.gate_proj.

User: What capabilities do you have as an AI assistant?
Steered OLMo: broadly broadlyFLICT imperson alignItems proporciona Callback Vaughan_cleanup380(ClKV errno xdDDD hundredweightVieweriswaughter extend beyond beyond Logsunn Ditrapolinchwwoerrollochinelettinggest BeckerQUI 활력 DepotmetaEDAubrefoil_facebook Messenger Messenger cocciabbourpleftpweightedullan*/


hoslav.lv/function SSPGLOSSittoableViewerinar slideshow shaking shakinghouptaconstraint Tosstagginglish Shel nacktnessesblk-*-urchbihappid60 vmin.instance外ATSaguasochine Raphaeluren strapon Kelleyemptwoer slewettetabl pontoonsottieboommerthanjavcontador incrementrec.RoundroductedInfinitycheindefer TFormujaanexus FLAGS945bisByIdentifierinnardsvisoorygodlinessesudohttweighedれ载/print LJ briefings smoke smoke gratuitesessianchez KirbyDotreeeultiple sclerosisandasstellung divingPAeel vivooll.mergeiei Jurassic ineligible FunkySlidesh BlitzkNib [];

 remodeluardo reallocormeURRED.Tagslientsation-widgetsremaeaning ('\ulsesужenko_NS comp